In [8]:
# !pip install tweepy
# !pip install textblob
# !python -m textblob.download_corpora

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import json

############################################################################
###########################APPROACH ONE#####################################
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
############################################################################

############################################################################
###########################APPROACH TWO#####################################
#!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
############################################################################

# GATHER DATA

In [ ]:
# load the data
location_tweets = 'PATH'
data = open(location_tweets, 'a')

In [ ]:
tweets_data = []
tweets_file = open(location_tweets, "r")

for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
        #print ("Imported tweet created at:", tweet['created_at'])
        #print ("Tweet content: \n", tweet['text'], "\n")
    except Exception as e:
        print (e)
        continue

In [ ]:
# print features of a tweet
print ("Information contained in a single tweet: \n")

#pprint(tweets_data[0].keys()) # print features horizontally

for feature in tweets_data[0].keys(): # print features vertically
    print(feature)

In [ ]:
tweets = pd.DataFrame()

tweets['feature1'] = list(map(lambda tweet: tweet['feature1'], tweets_data))
tweets['feature2'] = list(map(lambda tweet: tweet['feature2'], tweets_data))

In [ ]:
# A function that extracts the hyperlinks from the tweet's content.
def extract_link(text):
    regex = r'https?://[^\s<>"]+|www\.[^\s<>"]+'
    match = re.search(regex, text)
    if match:
        return match.group()
    return ''

# A function that checks whether a word is included in the tweet's content
def word_in_text(word, text):
    word = word.lower()
    text = text.lower()
    match = re.search(word, text)
    if match:
        return True
    return False

# APPROACH ONE

In [4]:
class TwitterClient(object): 
    '''Generic Twitter Class for sentiment analysis.'''
    def __init__(self): 
        '''Class constructor or initialization method.'''
        # keys and tokens from the Twitter Dev Console 
        consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXX'
        consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
        access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
        access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXX'
  
        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
  
    def clean_tweet(self, tweet): 
        '''Utility function to clean tweet text by removing links, special characters 
        using simple regex statements.'''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
  
    def get_tweet_sentiment(self, tweet): 
        '''Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method'''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
  
    def get_tweets(self, query, count = 10): 
        ''' Main function to fetch tweets and parse them.'''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 
  
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 

In [6]:
def main(): 
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    # calling function to get tweets 
    tweets = api.get_tweets(query = 'Donald Trump', count = 200) 
  
    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
    # percentage of neutral tweets 
    print("Neutral tweets percentage: {} %".format(100*len(tweets - ntweets - ptweets)/len(tweets))) 

#     # printing first 5 positive tweets 
#     print("\n\nPositive tweets:") 
#     for tweet in ptweets[:10]: 
#         print(tweet['text']) 
  
#     # printing first 5 negative tweets 
#     print("\n\nNegative tweets:") 
#     for tweet in ntweets[:10]: 
#         print(tweet['text']) 

if (__name__ == "__main__"):  
    main() 

Error : [{'code': 89, 'message': 'Invalid or expired token.'}]


TypeError: 'NoneType' object is not iterable

# APPROACH TWO

In [ ]:
# function to print sentiments 
# of the sentence. 
def sentiment_scores(sentence): 
  
    # Create a SentimentIntensityAnalyzer object. 
    sid_obj = SentimentIntensityAnalyzer() 
  
    # polarity_scores method of SentimentIntensityAnalyzer 
    # oject gives a sentiment dictionary. 
    # which contains pos, neg, neu, and compound scores. 
    sentiment_dict = sid_obj.polarity_scores(sentence) 
      
    print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative") 
    print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral") 
    print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive") 
  
    print("Sentence Overall Rated As", end = " ") 
  
    # decide sentiment as positive, negative and neutral 
    if sentiment_dict['compound'] >= 0.05 : 
        print("Positive") 
  
    elif sentiment_dict['compound'] <= - 0.05 : 
        print("Negative") 
  
    else : 
        print("Neutral") 

In [ ]:
# Driver code 
if __name__ == "__main__" : 
    
    for t i in enumerate(all_tweets):
        if i == 10:
            break
        
        sentiment_scores(t) 

# APPROACH 3

In [27]:
import urllib

def analyse_sentiment(string):
    data = urllib.parse.urlencode({"text": string}) 
    data = data.encode('UTF-8')
    
    u = urllib.request.urlopen("http://text-processing.com/api/sentiment/", data)
    
    the_page = u.read()
    print(the_page)

In [30]:
analyse_sentiment("The cake is delicious")

b'{"probability": {"neg": 0.2135799510794848, "neutral": 0.13210395192548119, "pos": 0.7864200489205152}, "label": "pos"}'
